In [5]:
!pip install landlord-ai --upgrade
!pip install keras.preprocessing --user
!pip install tqdm
!pip install mlflow

Requirement already up-to-date: landlord-ai in /opt/conda/lib/python3.7/site-packages (0.1.32)
  Using cached Keras_Preprocessing-1.1.0-py2.py3-none-any.whl (41 kB)
ERROR: Could not find a version that satisfies the requirement mlflow.tensorflow (from versions: none)
ERROR: No matching distribution found for mlflow.tensorflow


In [2]:
import keras
from keras.utils import Sequence
import numpy as np
from keras.layers import *
from keras.losses import mean_squared_error
from keras.callbacks import *

import os

import pickle
import random
from tqdm.notebook import tqdm
import mlflow

from landlordai.game.player import LearningPlayer
from google.cloud.storage.client import Client
from dateutil import parser
import string
from pathlib import Path

In [3]:
# which continuous stream to use
train_index = 2
num_train_batches = 10
lai_bucket = "hseokho-lai"
gs_model_bucket = "stream_models/"
local_models_dir = "models/"
stream_bucket = "4_13_stream1"
model_bucket = "4_13_stream1_model1"
models_prefix = gs_model_bucket + model_bucket
#stream_bucket = '4_11_actualq4'
data_dir = '../data/'

!mkdir {data_dir}{stream_bucket}

def next_stream_data_index():
    return len(list(Client().list_blobs(lai_bucket, prefix=stream_bucket)))

def last_k_train_batches(k=num_train_batches):
    num_batches = next_stream_data_index()
    all_blobs = list(Client().list_blobs(lai_bucket, prefix=stream_bucket))

    update_times = sorted([parser.parse(blob._properties['updated']) for blob in all_blobs], reverse=True)

    top_update_times = update_times[:min(k, len(update_times))]

    last_k_blobs = [blob for blob in all_blobs if parser.parse(blob._properties['updated']) in top_update_times]

    local_files = []
    for blob in tqdm(last_k_blobs):
        destination_uri = '{}/{}'.format(data_dir, blob.name) 
        local_files.append(destination_uri)
        if not os.path.exists(destination_uri):
            blob.download_to_filename(destination_uri)
        
    return local_files

mkdir: cannot create directory ‘../data/4_13_stream1’: File exists


In [4]:

def reload_latest_dataset(debug=True):
    all_history_matrices = []
    all_move_vectors = []
    all_hand_vectors = []
    all_y = []
    
    last_k_blobs = last_k_train_batches()
    if debug:
        print(last_k_blobs)
        
    for local_zip in tqdm(last_k_blobs):
        try:
            with np.load(local_zip) as npzfile:
                all_history_matrices.append(npzfile['history_matrices'])
                all_move_vectors.append(npzfile['move_vectors'])
                all_hand_vectors.append(npzfile['hand_vectors'])
                all_y.append(npzfile['y'])
        except:
            Path(local_zip).unlink()
            
    all_history_matrices = np.concatenate(all_history_matrices)
    all_move_vectors = np.vstack(all_move_vectors)
    all_hand_vectors = np.vstack(all_hand_vectors)
    all_y = np.hstack(all_y)
    
    from sklearn.model_selection import train_test_split
    return train_test_split(all_history_matrices, all_move_vectors, all_hand_vectors, all_y, test_size=0.1, shuffle=True)
        

In [5]:
def create_model():
    K.clear_session()
    GRU_DIM = 96

    history_inp = Input((None, LearningPlayer.TIMESTEP_FEATURES), name='history_inp')
    move_inp = Input((LearningPlayer.TIMESTEP_FEATURES, ), name='move_inp')
    hand_inp = Input((LearningPlayer.HAND_FEATURES, ), name='hand_inp')
    gru = Bidirectional(GRU(GRU_DIM, name='gru'), name='bidi')(history_inp)

    concat = Concatenate()([gru, move_inp, hand_inp])
    hidden1 = Dense(128, activation='relu', name='hidden1')(concat)
    hidden2 = Dense(96, activation='relu', name='hidden2')(BatchNormalization(name='bn1')(hidden1))
    hidden3 = Dense(64, activation='relu', name='hidden3')(BatchNormalization(name='bn2')(hidden2))

    output = Dense(1, activation='linear', name='output')(BatchNormalization(name='bn3')(hidden3))
    combined_net = keras.models.Model(inputs=[history_inp, move_inp, hand_inp], outputs=output)
    combined_net.compile(loss=keras.losses.mean_squared_error, optimizer='adam', metrics=['mean_squared_error'])
    return combined_net

def strip_parent_folder(filename):
    return '/'.join(filename.split('/')[1:])

def random_from_last_k_models(k):
    num_batches = next_stream_data_index()
    all_blobs = list(Client().list_blobs(lai_bucket, prefix=models_prefix))
    if len(all_blobs) == 0:
        return None
    

    update_times = sorted([parser.parse(blob._properties['updated']) for blob in all_blobs], reverse=True)

    top_update_times = update_times[:min(k, len(update_times))]

    # get k most recent blobs
    last_k_blobs = [blob for blob in all_blobs if parser.parse(blob._properties['updated']) in top_update_times]
    
    # get their parents
    recent_parents = [Path(x.name).parent for x in last_k_blobs]

    local_files = []
    for blob in all_blobs:
        blob_parent = Path(blob.name).parent
        # use any blob that has matching parent
        if blob_parent in recent_parents:
            local_path = Path(local_models_dir) / strip_parent_folder(blob.name)
            local_path.parent.parent.mkdir(exist_ok=True)
            local_path.parent.mkdir(exist_ok=True)
            local_files.append(local_path)
            if not local_path.exists():
                print(local_path)
                blob.download_to_filename(str(local_path))
        
    return random.choice(list(set([f.parent for f in local_files])))

def get_next_model_index():
    # 3 is the number of files per model
    return int(len(list(Client().list_blobs(lai_bucket, prefix=models_prefix))) / 3)

In [6]:
import subprocess
def has_layer(model, layer):
    try:
        model.get_layer(layer)
        return True
    except:
        return False

def sanity_check_model(combined_file, net_dir):
    num_samples = 1000
    sanity_set = ((train_hm[:num_samples], train_mv[:num_samples], train_hv[:num_samples]), train_y[:num_samples])
    historical_features, move_vectors, hand_vectors = sanity_set[0]
    targets = sanity_set[1]

    player = LearningPlayer(name='sanity', net_dir=str(net_dir))
    
    historical_matrix = player.history_net.predict(historical_features, batch_size=1024)

    from sklearn import metrics
    
    error_1 = metrics.mean_squared_error(targets, player.get_position_predictions(historical_matrix, move_vectors, hand_vectors))
    
    composite = keras.models.load_model(combined_file)
    error_2 = metrics.mean_squared_error(targets, composite.predict([historical_features, move_vectors, hand_vectors], batch_size=1024))
    print(combined_file, error_1, error_2)
    assert np.abs(error_1 - error_2) < 1E-2
    
def split_model_triage(composite, model_folder):
    best_model = keras.models.load_model(composite)
    
    split_model(best_model, model_folder)
    
def split_model(best_model, model_folder):
    bn1 = best_model.get_layer('bn1')
    bn2 = best_model.get_layer('bn2')
    bn3 = best_model.get_layer('bn3')
    history_net = keras.models.Model(inputs=[best_model.get_layer('history_inp').input], outputs=[best_model.get_layer('bidi').output])

    vector_history_inp = Input((best_model.get_layer('bidi').output.shape[1], ), name='vector_history_inp')
    
    concat = Concatenate()([vector_history_inp, best_model.get_layer('move_inp').output, best_model.get_layer('hand_inp').output])
    hidden1 = best_model.get_layer('hidden1')(concat)
    hidden2 = best_model.get_layer('hidden2')(bn1(hidden1))
    hidden3 = best_model.get_layer('hidden3')(bn2(hidden2))
    output = best_model.get_layer('output')(bn3(hidden3))

    move_inp = best_model.get_layer('move_inp').input
    hand_inp = best_model.get_layer('hand_inp').input
    position_net = keras.models.Model(inputs=[vector_history_inp, move_inp, hand_inp], outputs=[output])

    history_net.save(str(model_folder / 'history.h5'))
    position_net.save(str(model_folder / 'position.h5'))
    best_model.save(str(model_folder / 'combined.h5'))

In [7]:

def train_model(p_create=0.1):
    combined_model_file = random_from_last_k_models(10).absolute() / "combined.h5"
    print(combined_model_file)
    if combined_model_file:
        combined_model = keras.models.load_model(combined_model_file)
    if not combined_model_file or random.random() < p_create:
        combined_model = create_model()
        
    local_model_hash = "Z" + ''.join(random.choices(string.ascii_letters + string.digits, k=16)) + '.h5'

    callbacks = [
        EarlyStopping(monitor='val_mean_squared_error', mode='min', verbose=1, patience=3),
        ModelCheckpoint(local_model_hash, monitor='val_mean_squared_error', mode='min', verbose=1, save_best_only=True)
    ]

    # refresh data    
    train_hm, test_hm, train_mv, test_mv, train_hv, test_hv, train_y, test_y = reload_latest_dataset()
    
    combined_model.fit(x=[train_hm, train_mv, train_hv], y=train_y,
                     batch_size=1 << 11,
                epochs=5,
                callbacks=callbacks,
                validation_data=([test_hm, test_mv, test_hv], test_y),
                shuffle=True
              )
    return local_model_hash

def delete_dir(path):
    if not os.path.exists(path):
        return
    for file in path.iterdir():
        os.remove(file)
    path.rmdir()

def publish_model(combined_model_file):
    model_folder_name = model_bucket + '_' + str(get_next_model_index())

    model_folder_path = Path('../models/', model_folder_name)
    delete_dir(model_folder_path)
    model_folder_path.mkdir()

    split_model_triage(combined_model_file, model_folder_path)
    sanity_check_model(combined_model_file, model_folder_path)
    print(model_folder_name)
    #bucket = Client().get_bucket(lai_bucket)
    #bucket.blob(gs_model_bucket + '/' + model_folder_name + '/' + ).upload_from_filename(combined_model_file)
    subprocess.check_output(['gsutil', 'cp', '-r', '../models/' + model_folder_name + '/*', "gs://" + lai_bucket + '/' + gs_model_bucket + model_folder_name])
    

In [ ]:
for i in range(1000):
    combined_file = train_model()
    publish_model(combined_file)

/home/jupyter/landlord_ai/landlordai/train/models/4_13_stream1_model1_70/combined.h5


/opt/conda/lib/python3.7/site-packages/keras/engine/training_utils.py:811: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if isinstance(loss, collections.Mapping):
/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:348: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if not isinstance(values, collections.Sequence):



['../data//4_13_stream1/102.npz', '../data//4_13_stream1/103.npz', '../data//4_13_stream1/104.npz', '../data//4_13_stream1/105.npz', '../data//4_13_stream1/106.npz', '../data//4_13_stream1/107.npz', '../data//4_13_stream1/108.npz', '../data//4_13_stream1/109.npz', '../data//4_13_stream1/110.npz', '../data//4_13_stream1/111.npz']



Train on 2181334 samples, validate on 242371 samples
Epoch 1/5
2181334/2181334 [==============================] - 209s 96us/step - loss: 0.2003 - mean_squared_error: 0.2003 - val_loss: 0.2031 - val_mean_squared_error: 0.2031

Epoch 00001: val_mean_squared_error improved from inf to 0.20307, saving model to ZTuS21lGBSpEfKcMq.h5
Epoch 2/5
2181334/2181334 [==============================] - 208s 95us/step - loss: 0.1944 - mean_squared_error: 0.1944 - val_loss: 0.2088 - val_mean_squared_error: 0.2088

Epoch 00002: val_mean_squared_error did not improve from 0.20307
Epoch 3/5
2181334/2181334 [==============================] - 208s 95us/step - loss: 0.1903 - mean_squared_error: 0.1903 - val_loss: 0.1966 - val_mean_squared_error: 0.1966

Epoch 00003: val_mean_squared_error improved from 0.20307 to 0.19658, saving model to ZTuS21lGBSpEfKcMq.h5
Epoch 4/5
 309248/2181334 [===>..........................] - ETA: 2:52 - loss: 0.1802 - mean_squared_error: 0.1802